# Importações


In [67]:
import urllib
from itertools import product
from os import getenv
from sqlalchemy import create_engine
from dotenv import load_dotenv

import numpy as np
import pandas as pd
from datetime import datetime, timedelta, time
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
from fuzzywuzzy import process
from enum import Enum
import seaborn as sns

### Types


In [68]:
# cSpell: disable
class BSColorsEnum(Enum):

    DANGER_COLOR = "#dc3545"

    WARNING_COLOR = "#ffc107"

    SUCCESS_COLOR = "#198754"

    GREY_500_COLOR = "#adb5bd"

    GREY_600_COLOR = "#6c757d"

    GREY_700_COLOR = "#495057"

    GREY_800_COLOR = "#343a40"

    GREY_900_COLOR = "#212529"

    PRIMARY_COLOR = "#0d6efd"

    SECONDARY_COLOR = "#6c757d"

    INFO_COLOR = "#0dcaf0"

    GRAY_COLOR = "#adb5bd"

    TEAL_COLOR = "#20c997"

    ORANGE_COLOR = "#fd7e14"

    INDIGO_COLOR = "#6610f2"

    PINK_COLOR = "#d63384"

    PURPLE_COLOR = "#6f42c1"

    GREY_400_COLOR = "#ced4da"

    SPACE_CADET_COLOR = "#282f44"

    BLUE_DELFT_COLOR = "#0d6efd"


class IndicatorType(Enum):
    PERFORMANCE = "performance"
    REPAIR = "reparo"
    EFFICIENCY = "eficiencia"


def get_color(value, max_value):
    """
    Retorna uma cor hexadecimal com base no valor fornecido e no valor máximo.

    Parâmetros:
    value (float): O valor para o qual a cor será calculada.
    max_value (float): O valor máximo possível.

    Retorna:
    str: Uma cor hexadecimal correspondente ao valor fornecido.
    """

    # Cria um mapa de cores que vai do vermelho ao verde
    cmap = plt.get_cmap("RdYlGn")

    # Normaliza o valor para um número entre 0 e 1
    normalized_value = float(value) / max_value

    # Obtém a cor correspondente do mapa de cores
    rgba_color = cmap(normalized_value)

    # Converte a cor RGBA para uma string de cor hexadecimal
    hex_color = (
        f"#{int(rgba_color[0]*255):02x}{int(rgba_color[1]*255):02x}{int(rgba_color[2]*255):02x}"
    )

    return hex_color

# Database


## Conexão com o banco de dados


In [69]:
# database/connection.py

# cSpell: disable=invalid-name
load_dotenv()


class Connection:
    """
    Class Connection
    """

    def __init__(self):
        """
        Constructor

        Args:
            user (str): user
            password (str): password
            database (str): database
            driver (str): driver
            server (str): server

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        self.__user = getenv("PYMSSQL_USER")
        self.__password = getenv("PYMSSQL_PASSWORD")
        self.__database = getenv("PYMSSQL_DATABASE_AUTOMACAO")
        self.__database_totvsdb = getenv("PYMSSQL_DATABASE_TOTVSDB")
        self.__driver = "{ODBC Driver 17 for SQL Server}"
        self.__server = getenv("PYMSSQL_SERVER")

    def get_connection_automacao(self):
        """
        Get connection

        Returns:
            object: connection

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={self.__driver};"
                f"SERVER={self.__server};"
                f"DATABASE={self.__database};"
                f"UID={self.__user};"
                f"PWD={self.__password};"
            )
            # pylint: disable=consider-using-f-string
            conexao_automacao = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
            return conexao_automacao
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

    def get_connection_totvsdb(self):
        """
        Get connection

        Returns:
            object: connection

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={self.__driver};"
                f"SERVER={self.__server};"
                f"DATABASE={self.__database_totvsdb};"
                f"UID={self.__user};"
                f"PWD={self.__password};"
            )
            conexao_totvsdb = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", pool_size=2)
            return conexao_totvsdb
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

## Leitura do banco de dados


In [70]:
# database/db_read.py


# cSpell: disable=invalid-name
class Read(Connection):
    """
    Class Read
    Read data from the database and return a pandas dataframe
    Create query to be executed in the database
    """

    # pylint: disable=useless-super-delegation
    def __init__(self):
        """
        Constructor
        """
        super().__init__()

    def get_automacao_data(self, query: str) -> pd.DataFrame:
        """
        Get data from database AUTOMACAO and return a pandas dataframe.

        Parameters
        ----------
        query : str
            Query to be executed in the database

        Returns
        -------
        pandas dataframe
            Dataframe with the query result
        """
        try:
            connection = self.get_connection_automacao()
            data = pd.read_sql(query, connection)
            return data
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None
        finally:
            if connection:
                connection.dispose()

    def create_automacao_query(self, table: str, where: str = None, orderby: str = None) -> str:
        """
        Create query to be executed in the database AUTOMACAO.

        Parameters
        ----------
        table : str
            Table name
        where : str
            Where clause (optional)
        orderby : str
            Order by clause (optional)

        Returns
        -------
        str
            Query to be executed in the database
        """
        query = f"SELECT * FROM AUTOMACAO.dbo.{table}"

        if where:
            query += f" WHERE {where}"

        if orderby:
            query += f" ORDER BY {orderby}"

        return query

    def get_totvsdb_data(self, query: str) -> pd.DataFrame:
        """
        Retrieves data from the TotvsDB database using the provided SQL query.

        Args:
            query (str): The SQL query to execute.

        Returns:
            pd.DataFrame: A pandas DataFrame containing the retrieved data.

        Raises:
            Exception: If an error occurs while retrieving the data.

        """
        connection = None
        try:
            connection = self.get_connection_totvsdb()
            data = pd.read_sql(query, connection)
            return data
        # pylint: disable=broad-except
        except Exception as e:
            print(f"Erro ao buscar dados: {e}")
            return None
        finally:
            if connection:
                connection.dispose()

    def create_totvsdb_query(
        self, select: str, table: str, join: str = None, where: str = None, orderby: str = None
    ) -> str:
        """
        Creates a SQL query string for querying the TOTVS database.

        Args:
            select (str): The SELECT clause of the query.
            table (str): The table name to query.
            join (str, optional): The JOIN clause of the query. Defaults to None.
            where (str, optional): The WHERE clause of the query. Defaults to None.
            orderby (str, optional): The ORDER BY clause of the query. Defaults to None.

        Returns:
            str: The SQL query string.
        """
        query = f"SELECT {select} FROM {table}"

        if join:
            query += f" {join}"

        if where:
            query += f" WHERE {where}"

        if orderby:
            query += f" ORDER BY {orderby}"

        return query

## Query para o banco de dados


In [71]:
# // database/get_data.py
# cSpell: disable=invalid-name
class GetData:
    """
    Essa classe é responsável por realizar a leitura dos dados do banco de dados.
    É utilizada para fazer a leitura em segundo plano, sem que o usuário perceba.
    """

    def __init__(self):
        self.db_read = Read()

    def get_data(self) -> tuple:
        """
        Realiza a leitura dos dados do banco de dados.
        Retorna na ordem: df_ihm, df_info, df_cadastro
        """

        # Dia de hoje
        now = pd.to_datetime("today")

        # Encontrando primeiro dia do mês atual
        first_day = now.replace(day=1)

        # Mantendo apenas a data
        first_day = first_day.strftime("%Y-%m-%d")

        # Query para leitura dos dados de ocorrência
        query_ihm = self.db_read.create_automacao_query(
            table="maquina_ihm",
            where=f"data_registro >= '{first_day}'",
        )

        query_info = (
            "SELECT"
            " t1.maquina_id,"
            " (SELECT TOP 1 t2.linha FROM AUTOMACAO.dbo.maquina_cadastro t2"
            " WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro"
            " ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as linha,"
            " (SELECT TOP 1 t2.fabrica FROM AUTOMACAO.dbo.maquina_cadastro t2"
            " WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro"
            " ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as fabrica,"
            " t1.status,"
            " t1.turno,"
            " t1.contagem_total_ciclos,"
            " t1.contagem_total_produzido,"
            " t1.data_registro,"
            " t1.hora_registro"
            " FROM "
            " AUTOMACAO.dbo.maquina_info t1"
            f" WHERE data_registro >= '{first_day}'"
            " ORDER BY t1.data_registro DESC, t1.hora_registro DESC"
        )

        query_production = (
            "SELECT * "
            "FROM ( "
            "SELECT "
            "(SELECT TOP 1 t2.fabrica FROM AUTOMACAO.dbo.maquina_cadastro t2 "
            "WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro "
            "ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as fabrica, "
            "(SELECT TOP 1 t2.linha FROM AUTOMACAO.dbo.maquina_cadastro t2 "
            "WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro "
            "ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as linha, "
            "t1.maquina_id, "
            "t1.turno, "
            "t1.status, "
            "t1.contagem_total_ciclos as total_ciclos, "
            "t1.contagem_total_produzido as total_produzido, "
            "t1.data_registro, "
            "t1.hora_registro, "
            "ROW_NUMBER() OVER ( "
            "PARTITION BY t1.data_registro, t1.turno, t1.maquina_id "
            "ORDER BY t1.data_registro DESC, t1.hora_registro DESC"
            ") AS rn "
            "FROM AUTOMACAO.dbo.maquina_info t1 "
            ") AS t "
            f" WHERE rn = 1 AND data_registro >= '{first_day}' AND hora_registro > '00:01'"
            " ORDER BY data_registro DESC, linha"
        )

        # Leitura dos dados
        df_ihm = self.db_read.get_automacao_data(query_ihm)
        df_info = self.db_read.get_automacao_data(query_info)
        df_info_production = self.db_read.get_automacao_data(query_production)

        # Verificando se os dados foram lidos corretamente
        if df_ihm.empty or df_info.empty or df_info_production.empty:
            print("====== Erro na leitura dos dados ======")
            return None, None, None

        return df_ihm, df_info, df_info_production

    def get_maq_quality_data(self) -> pd.DataFrame:
        """
        Retrieves data from the 'maquina_qualidade' table in the AUTOMACAO database.

        Returns:
            DataFrame: A pandas DataFrame containing the retrieved data.
        """

        # Encontrando o primeiro dia de 6 meses atrás
        first_day = pd.to_datetime("today").replace(day=1).strftime("%Y-%m-%d")

        # Query para leitura dos dados de qualidade
        query = f"SELECT * FROM AUTOMACAO.dbo.qualidade_ihm WHERE data_registro >= '{first_day}'"

        df = self.db_read.get_automacao_data(query)

        return df


get_data = GetData()
df_ihm, df_info, df_info_production = get_data.get_data()
df_quality = get_data.get_maq_quality_data()

## Testes de saída do banco de dados


In [72]:
df_ihm.head(20)

,recno,linha,maquina_id,motivo,equipamento,problema,causa,os_numero,operador_id,data_registro,hora_registro
0,43,6,TMF001,Ajustes,Recheadora,Quantidade de Recheio não Conforme,Bico entupido,,000933,2024-05-07,08:55:24.653333
1,55,1,TMF003,Parada Programada,,Parada Planejada,Café e Ginástica Laboral,,33,2024-05-07,09:07:40.426666
2,62,1,TMF003,Ajustes,Armadora de Caixas,Caixa não Selando,Ajuste na Dimensão da Caixa,,33,2024-05-07,10:14:30.766666
3,63,1,TMF003,Ajustes,Armadora de Caixas,Caixa não Selando,Ajuste na Dimensão da Caixa,,33,2024-05-07,10:46:15.656666
4,105,1,TMF003,Parada Programada,,Parada Planejada,Refeição,,33,2024-05-07,18:58:55.336666
5,106,2,TMF011,Parada Programada,,Parada Planejada,Refeição,,001643,2024-05-07,19:02:28.570000
6,107,6,TMF001,Manutenção,Termoformadora,Bandeja Murcha ou Cheia,Realizar análise de falha,066636,000933,2024-05-07,20:44:47.370000
7,108,6,TMF001,Manutenção,Termoformadora,Bandeja Murcha ou Cheia,Realizar análise de falha,,000933,2024-05-07,20:50:29.923333
8,109,6,TMF001,Parada Programada,,Parada Planejada,Refeição,,000933,2024-05-08,05:07:02.910000
9,111,6,TMF001,Ajustes,Termoformadora,Bobina desenquadrando na vertical,Ajuste de Fotocélula,,000933,2024-05-08,08:55:40.236666


In [73]:
df_info.head(20)

,maquina_id,linha,fabrica,status,turno,contagem_total_ciclos,contagem_total_produzido,data_registro,hora_registro
0,TMF005,3.0,1,false,MAT,3374.0,3300.0,2024-05-28,11:06:08.426666
1,TMF011,2.0,1,false,MAT,3578.0,3536.0,2024-05-28,11:06:07.423333
2,TMF003,1.0,1,false,MAT,2472.0,2472.0,2024-05-28,11:06:06.423333
3,TMF004,9.0,1,false,MAT,0.0,0.0,2024-05-28,11:04:14.430000
4,TMF014,8.0,1,true,MAT,3206.0,2.0,2024-05-28,11:04:13.426666
5,TMF006,7.0,1,true,MAT,3692.0,3668.0,2024-05-28,11:04:12.426666
6,TMF001,6.0,1,false,MAT,3968.0,3740.0,2024-05-28,11:04:11.426666
7,TMF002,5.0,1,true,MAT,1096.0,892.0,2024-05-28,11:04:10.426666
8,TMF012,4.0,1,false,MAT,3436.0,3390.0,2024-05-28,11:04:09.423333
9,TMF005,3.0,1,false,MAT,3374.0,3300.0,2024-05-28,11:04:08.420000


In [74]:
df_info_production

,fabrica,linha,maquina_id,turno,status,total_ciclos,total_produzido,data_registro,hora_registro,rn
0,1,1.0,TMF003,NOT,false,6468.0,6468.0,2024-05-28,07:58:05.893333,1
1,1,1.0,TMF003,MAT,false,2472.0,2472.0,2024-05-28,11:06:06.423333,1
2,1,2.0,TMF011,NOT,false,6474.0,6416.0,2024-05-28,07:58:06.896666,1
3,1,2.0,TMF011,MAT,false,3578.0,3536.0,2024-05-28,11:06:07.423333,1
4,1,3.0,TMF005,MAT,false,3374.0,3300.0,2024-05-28,11:06:08.426666,1
...,...,...,...,...,...,...,...,...,...,...
898,2,13.0,TMF013,NOT,false,24.0,0.0,2024-05-02,07:58:09.200000,1
899,2,13.0,TMF013,VES,false,0.0,0.0,2024-05-02,23:58:37.856666,1
900,2,14.0,TMF009,NOT,false,0.0,0.0,2024-05-02,07:58:10.200000,1
901,2,14.0,TMF009,VES,false,0.0,0.0,2024-05-02,23:58:38.856666,1


In [75]:
df_quality.head(20)

,recno,linha,maquina_id,bdj_vazias,bdj_retrabalho,descarte_paes,descarte_paes_pasta,descarte_pasta,data_registro,hora_registro
0,14,6,TMF001,0,0,0.0,0,0.00,2024-05-22,06:33:27.346666
1,15,6,TMF001,0,0,0.0,0,0.00,2024-05-23,04:41:41.660000
2,16,1,TMF003,0,0,0.0,0,0.00,2024-05-24,13:34:56.946666
3,17,2,TMF011,0,0,0.0,0,0.00,2024-05-24,13:37:33.323333
4,18,3,TMF005,0,0,0.0,0,0.00,2024-05-24,13:38:07.723333
5,19,4,TMF012,0,0,0.0,0,0.00,2024-05-24,13:39:07.796666
6,20,5,TMF002,0,0,0.0,0,0.00,2024-05-24,13:39:40.593333
7,21,6,TMF001,0,0,0.0,0,0.00,2024-05-24,13:40:27.533333
8,22,6,TMF001,0,0,0.0,0,0.00,2024-05-24,13:41:41.526666
9,23,7,TMF006,0,0,0.0,0,0.00,2024-05-24,13:42:24.526666


# Limpeza de dados e análise exploratória


## Análise de dados - Clean Data


In [76]:
# service/clean_data.py


# cSpell: disable=invalid-name
class CleanData:
    """
    Essa classe é responsável por limpar os dados lidos do banco de dados.
    """

    def __init__(self, df_ihm, df_info, df_info_production):
        self.df_ihm = df_ihm
        self.df_info = df_info
        self.df_info_production = df_info_production

    @staticmethod
    def __clean_basics(df: pd.DataFrame) -> pd.DataFrame:
        """
        Limpa os dados básicos de um DataFrame.
        """

        # Remove valores duplicados
        df = df.drop_duplicates()

        # Remove linhas com valores nulos
        df = df.dropna(subset=["maquina_id", "data_registro", "hora_registro"])

        # Converte a coluna 'hora_registro' para string e remove os milissegundos
        df["hora_registro"] = df["hora_registro"].astype(str).str.split(".").str[0]

        # Converte as colunas 'data_registro' e 'hora_registro' para datetime
        df["data_registro"] = pd.to_datetime(df["data_registro"])
        df["hora_registro"] = pd.to_datetime(df["hora_registro"], format="%H:%M:%S").dt.time

        # Substitui os valores NaN por 0 antes de converter para int
        df["linha"] = df["linha"].fillna(0).astype(int)

        # Coluna linha para int
        df["linha"] = df["linha"].astype(int)

        # Remover onde a 'linha' é 0
        df = df[df["linha"] != 0]

        return df

    @staticmethod
    def __clean_info(df: pd.DataFrame) -> pd.DataFrame:
        """
        Limpa os dados de informações.
        Fica desatualizado após implementação das IHMs.
        Manter apenas para código legado.
        """

        # Reordenar o dataframe
        df = df.sort_values(by=["maquina_id", "data_registro", "hora_registro"])

        # Remover a primeira entrada caso seja do turno "VES"
        mask = (df["turno"] == "VES") & (df["maquina_id"] != df["maquina_id"].shift())
        df = df[~mask]

        # Ajustar o turno VES caso a hora da entrada passe de 00:00, para 23:59 do dia anterior
        mask = (
            (df["turno"] == "VES")
            & (df["hora_registro"] > time(0, 0, 0))
            & (df["hora_registro"] < time(0, 5, 0))
        )

        df.loc[mask, "hora_registro"] = time(23, 59, 59)
        df.loc[mask, "data_registro"] = df.loc[mask, "data_registro"] - timedelta(days=1)

        # Reordenar o dataframe
        df = df.sort_values(by=["linha", "data_registro", "hora_registro"])

        # Reiniciar o índice
        df = df.reset_index(drop=True)

        return df

    @staticmethod
    def __clean_prod(df: pd.DataFrame) -> pd.DataFrame:
        """
        Limpa os dados de produção.
        """

        # Coluna auxiliar para ordenar os turnos
        df["turno_aux"] = df["turno"].map({"MAT": 2, "VES": 3, "NOT": 1})

        # Ordenar os valores
        df = df.sort_values(by=["linha", "data_registro", "turno_aux"])

        # Remover a coluna auxiliar
        df = df.drop(columns=["turno_aux", "rn", "status"])

        # Coluna linha para int
        df["linha"] = df["linha"].astype(int)

        # Reiniciar o índice
        df = df.reset_index(drop=True)

        return df

    def clean_data(self) -> tuple:
        """
        Limpa os dados lidos do banco de dados.
        Retorna na ordem: df_ihm, df_info, df_info_production
        """

        # Limpando os dados(removendo linhas duplicadas ou nulas em informações importantes)
        self.df_ihm = self.__clean_basics(self.df_ihm)
        self.df_info = self.__clean_basics(self.df_info)
        self.df_info_production = self.__clean_basics(self.df_info_production)

        # Ajustando o status
        # Ajusta nomenclatura do status
        self.df_info["status"] = np.where(self.df_info["status"] == "true", "rodando", "parada")

        # Ajustes nas informações de produção
        self.df_info_production = self.__clean_prod(self.df_info_production)

        # NOTE: A fonção abaixo ficará desatualizada após a implementação das IHMs
        self.df_info = self.__clean_info(self.df_info)

        return self.df_ihm, self.df_info, self.df_info_production


clean_data = CleanData(df_ihm.copy(), df_info.copy(), df_info_production.copy())
df_ihm_cleaned, df_info_cleaned, df_info_production_cleaned = clean_data.clean_data()

### Saída de dados limpos


In [77]:
df_ihm_cleaned

,recno,linha,maquina_id,motivo,equipamento,problema,causa,os_numero,operador_id,data_registro,hora_registro
0,43,6,TMF001,Ajustes,Recheadora,Quantidade de Recheio não Conforme,Bico entupido,,000933,2024-05-07,08:55:24
1,55,1,TMF003,Parada Programada,,Parada Planejada,Café e Ginástica Laboral,,33,2024-05-07,09:07:40
2,62,1,TMF003,Ajustes,Armadora de Caixas,Caixa não Selando,Ajuste na Dimensão da Caixa,,33,2024-05-07,10:14:30
3,63,1,TMF003,Ajustes,Armadora de Caixas,Caixa não Selando,Ajuste na Dimensão da Caixa,,33,2024-05-07,10:46:15
4,105,1,TMF003,Parada Programada,,Parada Planejada,Refeição,,33,2024-05-07,18:58:55
...,...,...,...,...,...,...,...,...,...,...,...
471,611,3,TMF005,Fluxo,Termoformadora,Esteira Cheia,Robô parado,0,933,2024-05-28,10:11:03
472,612,2,TMF011,Parada Programada,6,Parada Planejada,Refeição,0,1815,2024-05-28,10:57:08
473,613,1,TMF003,Parada Programada,,Parada Planejada,Refeição,0,1815,2024-05-28,10:57:39
474,614,4,TMF012,Parada Programada,,Parada Planejada,Refeição,0,293,2024-05-28,11:05:03


In [78]:
df_info_cleaned

,maquina_id,linha,fabrica,status,turno,contagem_total_ciclos,contagem_total_produzido,data_registro,hora_registro
0,TMF003,1,1,parada,NOT,10.0,10.0,2024-05-02,00:01:10
1,TMF003,1,1,parada,NOT,10.0,10.0,2024-05-02,00:03:10
2,TMF003,1,1,parada,NOT,10.0,10.0,2024-05-02,00:05:10
3,TMF003,1,1,parada,NOT,10.0,10.0,2024-05-02,00:07:10
4,TMF003,1,1,parada,NOT,10.0,10.0,2024-05-02,00:09:10
...,...,...,...,...,...,...,...,...,...
210923,TMF009,14,2,parada,MAT,0.0,0.0,2024-05-28,10:55:29
210924,TMF009,14,2,parada,MAT,0.0,0.0,2024-05-28,10:57:29
210925,TMF009,14,2,parada,MAT,0.0,0.0,2024-05-28,10:59:29
210926,TMF009,14,2,parada,MAT,0.0,0.0,2024-05-28,11:01:29


In [79]:
df_info_production_cleaned

,fabrica,linha,maquina_id,turno,total_ciclos,total_produzido,data_registro,hora_registro
0,1,1,TMF003,NOT,6398.0,6274.0,2024-05-02,07:59:12
1,1,1,TMF003,MAT,2124.0,2116.0,2024-05-02,15:59:13
2,1,1,TMF003,VES,6050.0,5960.0,2024-05-02,23:59:15
3,1,1,TMF003,NOT,8728.0,8712.0,2024-05-03,07:59:16
4,1,1,TMF003,MAT,4042.0,4000.0,2024-05-03,15:59:17
...,...,...,...,...,...,...,...,...
883,2,14,TMF009,NOT,0.0,0.0,2024-05-27,07:59:24
884,2,14,TMF009,MAT,0.0,0.0,2024-05-27,15:59:26
885,2,14,TMF009,VES,0.0,0.0,2024-05-27,23:59:27
886,2,14,TMF009,NOT,0.0,0.0,2024-05-28,07:59:29


# Unindo os dados


## IHM + Info


In [80]:
class JoinInfoIHM:
    """
    Essa classe é responsável por juntar os DataFrames de informações e IHM.
    """

    def __init__(self, df_ihm, df_info):
        self.df_ihm = df_ihm
        self.df_info = df_info

    def join_data(self) -> pd.DataFrame:
        """
        Junta os DataFrames de informações e IHM.
        """

        # Cria uma coluna auxiliar com data e hora para fazer o merge
        self.df_info["data_hora"] = pd.to_datetime(
            self.df_info["data_registro"].astype(str)
            + " "
            + self.df_info["hora_registro"].astype(str)
        )

        self.df_ihm["data_hora"] = pd.to_datetime(
            self.df_ihm["data_registro"].astype(str)
            + " "
            + self.df_ihm["hora_registro"].astype(str)
        )

        # Classifica os DataFrames
        self.df_info = self.df_info.sort_values(by="data_hora")
        self.df_ihm = self.df_ihm.sort_values(by="data_hora")

        # Junta os DataFrames
        df = pd.merge_asof(
            self.df_info,
            self.df_ihm.drop(columns="recno"),
            on="data_hora",
            by="maquina_id",
            direction="nearest",
            tolerance=pd.Timedelta("1 min 10 s"),
        )

        # Ajustas os tipos das colunas contagem_total_ciclos e contagem_total_produzido para int
        df["contagem_total_ciclos"] = df["contagem_total_ciclos"].astype("Int64")
        df["contagem_total_produzido"] = df["contagem_total_produzido"].astype("Int64")

        # Reordenar as colunas mantendo só as colunas necessárias e renomeando as colunas para facilitar a compreensão
        df = df[
            [
                "fabrica",
                "linha_x",
                "maquina_id",
                "turno",
                "status",
                "contagem_total_ciclos",
                "contagem_total_produzido",
                "data_registro_x",
                "hora_registro_x",
                "motivo",
                "equipamento",
                "problema",
                "causa",
                "os_numero",
                "operador_id",
                "data_registro_y",
                "hora_registro_y",
            ]
        ]

        df.columns = [
            "fabrica",
            "linha",
            "maquina_id",
            "turno",
            "status",
            "contagem_total_ciclos",
            "contagem_total_produzido",
            "data_registro",
            "hora_registro",
            "motivo",
            "equipamento",
            "problema",
            "causa",
            "os_numero",
            "operador_id",
            "data_registro_ihm",
            "hora_registro_ihm",
        ]

        # Reordenar pela linha, data_registro_info e hora_registro_info
        df = df.sort_values(by=["linha", "data_registro", "hora_registro"])

        return df


join_info_ihm = JoinInfoIHM(df_ihm_cleaned.copy(), df_info_cleaned.copy())
df_joined = join_info_ihm.join_data()

### Saída de Join


In [81]:
df_joined

,fabrica,linha,maquina_id,turno,status,contagem_total_ciclos,contagem_total_produzido,data_registro,hora_registro,motivo,equipamento,problema,causa,os_numero,operador_id,data_registro_ihm,hora_registro_ihm
5,1,1,TMF003,NOT,parada,10,10,2024-05-02,00:01:10,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
18,1,1,TMF003,NOT,parada,10,10,2024-05-02,00:03:10,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
31,1,1,TMF003,NOT,parada,10,10,2024-05-02,00:05:10,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
44,1,1,TMF003,NOT,parada,10,10,2024-05-02,00:07:10,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
57,1,1,TMF003,NOT,parada,10,10,2024-05-02,00:09:10,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210859,2,14,TMF009,MAT,parada,0,0,2024-05-28,10:55:29,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
210873,2,14,TMF009,MAT,parada,0,0,2024-05-28,10:57:29,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
210887,2,14,TMF009,MAT,parada,0,0,2024-05-28,10:59:29,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
210901,2,14,TMF009,MAT,parada,0,0,2024-05-28,11:01:29,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


# Service


## Info + IHM Service


In [82]:
class ServiceInfoIHM:
    """
    Essa classe é responsável por realizar a leitura, limpeza e junção dos dados de informações e IHM.
    """

    def __init__(self):
        self.get_data = GetData()
        self.clean_data = CleanData
        self.join_info_ihm = JoinInfoIHM
        self.data = self.get_data.get_data()

    def __get_info_ihm(self) -> pd.DataFrame:
        """
        Realiza a leitura, limpeza e junção dos dados de informações e IHM.
        """

        # Realiza a leitura dos dados
        df_ihm, df_info, _ = self.data

        # Verifica se os dados foram lidos corretamente
        if df_ihm is None or df_info is None or df_info_production is None:
            return None

        # Limpa os dados
        clean_data = self.clean_data(df_ihm, df_info, df_info_production)
        df_ihm_cleaned, df_info_cleaned, _ = clean_data.clean_data()

        # Junta os dados
        join_info_ihm = self.join_info_ihm(df_ihm_cleaned, df_info_cleaned)
        df_joined = join_info_ihm.join_data()

        return df_joined

    def __identify_changes(self, df, column) -> pd.Series:
        return df[column].ne(df[column].shift())

    def __status_change(self, df):
        # Checa se houve mudança de status, maquina_id e turno
        columns_to_check = ["status", "maquina_id", "turno"]
        for column in columns_to_check:
            df[f"{column}_change"] = self.__identify_changes(df, column)

        # Coluna auxiliar que identifica se houve alguma mudança
        df["change"] = df[["status_change", "maquina_id_change", "turno_change"]].any(axis=1)

        return df

    def __fill_occ(self, df):
        # Preenche os valores nulos de paradas
        df["motivo"] = df.groupby("group")["motivo"].transform(lambda x: x.ffill().bfill())
        df["equipamento"] = df.groupby("group")["equipamento"].transform(
            lambda x: x.ffill().bfill()
        )
        df["problema"] = df.groupby("group")["problema"].transform(lambda x: x.ffill().bfill())
        df["causa"] = df.groupby("group")["causa"].transform(lambda x: x.ffill().bfill())
        df["os_numero"] = df.groupby("group")["os_numero"].transform(lambda x: x.ffill().bfill())
        df["operador_id"] = df.groupby("group")["operador_id"].transform(
            lambda x: x.ffill().bfill()
        )
        df["data_registro_ihm"] = df.groupby("group")["data_registro_ihm"].transform(
            lambda x: x.ffill().bfill()
        )
        df["hora_registro_ihm"] = df.groupby("group")["hora_registro_ihm"].transform(
            lambda x: x.ffill().bfill()
        )

        # Se os dado de uma coluna for '' ou ' ', substituir por NaN
        df = df.replace(r"^s*$", None, regex=True)
        # O ^ indica o início de uma string, o $ indica o fim de uma string, e s* zero ou mais espaços em branco

        return df

    def __group_and_calc_time(self, df):
        # Agrupa as mudanças
        df = (
            df.groupby(["group"])
            .agg(
                fabrica=("fabrica", "first"),
                linha=("linha", "first"),
                maquina_id=("maquina_id", "first"),
                turno=("turno", "first"),
                status=("status", "first"),
                data_registro=("data_registro", "first"),
                hora_registro=("hora_registro", "first"),
                motivo=("motivo", "first"),
                equipamento=("equipamento", "first"),
                problema=("problema", "first"),
                causa=("causa", "first"),
                os_numero=("os_numero", "first"),
                operador_id=("operador_id", "first"),
                data_registro_ihm=("data_registro_ihm", "first"),
                hora_registro_ihm=("hora_registro_ihm", "first"),
                data_hora=("data_hora", "first"),
                change=("change", "first"),
                maquina_id_change=("maquina_id_change", "first"),
                change_date=("change_date", "first"),
                motivo_change=("motivo_change", "first"),
            )
            .reset_index()
        )

        # Nova coluna com a data_hora_final do status/parada
        df["data_hora_final"] = (
            df.groupby("maquina_id")["data_hora"].shift(-1).where(~df["maquina_id_change"])
        )

        # Atualiza a hora final caso mude a máquina
        mask = df["maquina_id_change"]
        df["data_hora_final"] = np.where(
            mask,
            df["change_date"].shift(-1),
            df["data_hora_final"],
        )

        # Caso a data_hora_final seja nula, remove a linha
        df = df.dropna(subset=["data_hora_final"]).reset_index(drop=True)

        # Calcula o tempo de cada status
        df["tempo"] = (
            pd.to_datetime(df["data_hora_final"]) - pd.to_datetime(df["data_hora"])
        ).dt.total_seconds() / 60

        # Arredondar e converter para inteiro
        df["tempo"] = df["tempo"].round().astype(int)

        return df

    def get_info_ihm_adjusted(self) -> pd.DataFrame:
        """
        Realiza a leitura, limpeza e junção dos dados de informações e IHM.
        Ajusta os dados para o formato correto.
        """

        # Realiza a leitura, limpeza e junção dos dados
        df_joined = self.__get_info_ihm()

        if df_joined is None:
            return None

        # ========================= Identifica Onde Há Mudança De Status ========================= #

        df_joined = self.__status_change(df_joined)

        # ========================= Preenchendo Valores Nulos De Paradas ========================= #

        # Cria um grupo para cada mudança de status
        df_joined["group"] = df_joined["change"].cumsum()

        # Preenche os valores nulos de paradas
        df_joined = self.__fill_occ(df_joined)

        # Coluna auxiliar para identificar mudança na coluna motivo, se não for nula
        mask = (df_joined["motivo"].ne(df_joined["motivo"].shift())) | (
            df_joined["causa"].ne(df_joined["causa"].shift())
        )
        df_joined["motivo_change"] = mask & df_joined["motivo"].notnull()

        # Atualiza o change
        df_joined["change"] = df_joined["change"] | df_joined["motivo_change"]

        # Refaz o grupo para considerar a mudança na coluna motivo
        df_joined["group"] = df_joined["change"].cumsum()

        # Coluna auxiliar para unir data e hora
        df_joined["data_hora"] = pd.to_datetime(
            df_joined["data_registro"].astype(str) + " " + df_joined["hora_registro"].astype(str)
        )

        # Coluna auxiliar para identificar a data/hora da mudança
        df_joined["change_date"] = (
            df_joined.groupby("maquina_id")["data_hora"].shift(0).where(df_joined["change"])
        )

        # ============================ Calcula O Tempo Parada Ou Rodando ========================== #

        # Calcula os tempos
        df_joined = self.__group_and_calc_time(df_joined)

        # Ajustar status para levar em conta testes
        mask = (
            (df_joined["status"] == "rodando")
            & (df_joined["tempo"] < 10)
            & (df_joined["turno"].eq(df_joined["turno"].shift(-1)))
        )

        df_joined["status"] = np.where(mask, "parada", df_joined["status"])

        # Ajuste em change para refletir alterações
        df_joined = self.__status_change(df_joined)
        df_joined["change"] = df_joined["change"] | df_joined["motivo_change"]

        # Refaz o group
        df_joined["group"] = df_joined["change"].cumsum()

        # Preenche os valores nulos de paradas
        df_joined = self.__fill_occ(df_joined)

        # Recalcula os tempos
        df_joined = self.__group_and_calc_time(df_joined)

        # Se o tempo for negativo, ajustar para 0
        df_joined["tempo"] = df_joined["tempo"].clip(lower=0)

        # Se o tempo for maior que 480 minutos, ajustar para 480
        df_joined["tempo"] = df_joined["tempo"].clip(upper=480)

        # Remove colunas auxiliares
        df_joined = df_joined.drop(
            columns=[
                "maquina_id_change",
                "change",
                "maquina_id_change",
                "change_date",
                "motivo_change",
                "group",
            ]
        )

        return df_joined

    def get_time_working(self, df: pd.DataFrame) -> pd.Series:
        """
        Calcula o tempo de trabalho de cada máquina.
        """

        # Filtra apenas as máquinas que estão rodando
        df = df[df["status"] == "rodando"]

        # Agrupa por máquina e turno
        df = (
            df.groupby(["maquina_id", "linha", "turno", "data_registro", "status"], observed=False)[
                "tempo"
            ]
            .sum()
            .reset_index()
        )

        # Renomear coluna
        df = df.rename(columns={"status": "motivo"})

        # Capitaliza o motivo
        df["motivo"] = df["motivo"].str.capitalize()

        return df

    def get_maq_stopped(self, df: pd.DataFrame) -> pd.Series:

        # Filtra apenas as máquinas que estão paradas
        df = df[df["status"] == "parada"]

        # Reinicia o índice
        df = df.reset_index(drop=True)

        return df


service_info_ihm = ServiceInfoIHM()

## Saídas do serviço


In [83]:
df_info_ihm = service_info_ihm.get_info_ihm_adjusted()
df_info_ihm

C:\Users\bruno.thomaz\AppData\Local\Temp\ipykernel_1172\74399151.py:50: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\bruno.thomaz\AppData\Local\Temp\ipykernel_1172\74399151.py:52: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\bruno.thomaz\AppData\Local\Temp\ipykernel_1172\74399151.py:54: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcast

,fabrica,linha,maquina_id,turno,status,data_registro,hora_registro,motivo,equipamento,problema,causa,os_numero,operador_id,data_registro_ihm,hora_registro_ihm,data_hora,data_hora_final,tempo
0,1,1,TMF003,NOT,parada,2024-05-02,00:01:10,None,None,None,None,None,None,NaT,None,2024-05-02 00:01:10,2024-05-02 00:21:11,20
1,1,1,TMF003,NOT,rodando,2024-05-02,00:21:11,None,None,None,None,None,None,NaT,None,2024-05-02 00:21:11,2024-05-02 00:33:11,12
2,1,1,TMF003,NOT,parada,2024-05-02,00:33:11,None,None,None,None,None,None,NaT,None,2024-05-02 00:33:11,2024-05-02 01:17:11,44
3,1,1,TMF003,NOT,rodando,2024-05-02,01:17:11,None,None,None,None,None,None,NaT,None,2024-05-02 01:17:11,2024-05-02 02:09:11,52
4,1,1,TMF003,NOT,parada,2024-05-02,02:09:11,None,None,None,None,None,None,NaT,None,2024-05-02 02:09:11,2024-05-02 02:19:11,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7494,2,14,TMF009,MAT,parada,2024-05-26,08:01:20,None,None,None,None,None,None,NaT,None,2024-05-26 08:01:20,2024-05-26 16:01:21,480
7495,2,14,TMF009,VES,parada,2024-05-26,16:01:21,None,None,None,None,None,None,NaT,None,2024-05-26 16:01:21,2024-05-27 00:01:23,480
7496,2,14,TMF009,NOT,parada,2024-05-27,00:01:23,None,None,None,None,None,None,NaT,None,2024-05-27 00:01:23,2024-05-27 08:01:24,480
7497,2,14,TMF009,MAT,parada,2024-05-27,08:01:24,None,None,None,None,None,None,NaT,None,2024-05-27 08:01:24,2024-05-27 16:01:26,480


In [84]:
df_time_working = service_info_ihm.get_time_working(df_info_ihm.copy())
df_time_working

,maquina_id,linha,turno,data_registro,motivo,tempo
0,TMF001,6,MAT,2024-05-02,Rodando,400
1,TMF001,6,MAT,2024-05-03,Rodando,346
2,TMF001,6,MAT,2024-05-04,Rodando,322
3,TMF001,6,MAT,2024-05-06,Rodando,395
4,TMF001,6,MAT,2024-05-07,Rodando,272
...,...,...,...,...,...,...
660,TMF014,8,VES,2024-05-21,Rodando,101
661,TMF014,8,VES,2024-05-22,Rodando,304
662,TMF014,8,VES,2024-05-23,Rodando,128
663,TMF014,8,VES,2024-05-26,Rodando,194


In [85]:
df_maq_stopped = service_info_ihm.get_maq_stopped(df_info_ihm.copy())
df_maq_stopped

,fabrica,linha,maquina_id,turno,status,data_registro,hora_registro,motivo,equipamento,problema,causa,os_numero,operador_id,data_registro_ihm,hora_registro_ihm,data_hora,data_hora_final,tempo
0,1,1,TMF003,NOT,parada,2024-05-02,00:01:10,None,None,None,None,None,None,NaT,None,2024-05-02 00:01:10,2024-05-02 00:21:11,20
1,1,1,TMF003,NOT,parada,2024-05-02,00:33:11,None,None,None,None,None,None,NaT,None,2024-05-02 00:33:11,2024-05-02 01:17:11,44
2,1,1,TMF003,NOT,parada,2024-05-02,02:09:11,None,None,None,None,None,None,NaT,None,2024-05-02 02:09:11,2024-05-02 02:19:11,10
3,1,1,TMF003,NOT,parada,2024-05-02,02:47:11,None,None,None,None,None,None,NaT,None,2024-05-02 02:47:11,2024-05-02 04:01:11,74
4,1,1,TMF003,NOT,parada,2024-05-02,04:33:11,None,None,None,None,None,None,NaT,None,2024-05-02 04:33:11,2024-05-02 04:53:11,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3900,2,14,TMF009,MAT,parada,2024-05-26,08:01:20,None,None,None,None,None,None,NaT,None,2024-05-26 08:01:20,2024-05-26 16:01:21,480
3901,2,14,TMF009,VES,parada,2024-05-26,16:01:21,None,None,None,None,None,None,NaT,None,2024-05-26 16:01:21,2024-05-27 00:01:23,480
3902,2,14,TMF009,NOT,parada,2024-05-27,00:01:23,None,None,None,None,None,None,NaT,None,2024-05-27 00:01:23,2024-05-27 08:01:24,480
3903,2,14,TMF009,MAT,parada,2024-05-27,08:01:24,None,None,None,None,None,None,NaT,None,2024-05-27 08:01:24,2024-05-27 16:01:26,480


# Dados de Saída


In [86]:
print(f"Maquina Parada --> Linhas x Colunas: {df_maq_stopped.shape}")
print("---------------------------------------")
print(
    f"Maquina Parada --> Memória Ocupada: {df_maq_stopped.memory_usage().sum() / 1024 ** 2:.2f} MB"
)
print("---------------------------------------")
print(f"Maquina Parada --> Tipos:\n{df_maq_stopped.dtypes}")

Maquina Parada --> Linhas x Colunas: (3905, 18)
---------------------------------------
Maquina Parada --> Memória Ocupada: 0.51 MB
---------------------------------------
Maquina Parada --> Tipos:
fabrica                      object
linha                         int32
maquina_id                   object
turno                        object
status                       object
data_registro        datetime64[ns]
hora_registro                object
motivo                       object
equipamento                  object
problema                     object
causa                        object
os_numero                    object
operador_id                  object
data_registro_ihm    datetime64[ns]
hora_registro_ihm            object
data_hora            datetime64[ns]
data_hora_final      datetime64[ns]
tempo                         int32
dtype: object


In [87]:
print(f"Tempo de Trabalho --> Linhas x Colunas: {df_time_working.shape}")
print("---------------------------------------")
print(
    f"Tempo de Trabalho --> Memória Ocupada: {df_time_working.memory_usage().sum() / 1024 ** 2:.2f} MB"
)
print("---------------------------------------")
print(f"Tempo de Trabalho --> Tipos:\n{df_time_working.dtypes}")

Tempo de Trabalho --> Linhas x Colunas: (665, 6)
---------------------------------------
Tempo de Trabalho --> Memória Ocupada: 0.03 MB
---------------------------------------
Tempo de Trabalho --> Tipos:
maquina_id               object
linha                     int32
turno                    object
data_registro    datetime64[ns]
motivo                   object
tempo                     int32
dtype: object


In [88]:
print(f"Informações --> Linhas x Colunas: {df_info_ihm.shape}")
print("---------------------------------------")
print(f"Informações --> Memória Ocupada: {df_info_ihm.memory_usage().sum() / 1024 ** 2:.2f} MB")
print("---------------------------------------")
print(f"Informações --> Tipos:\n{df_info_ihm.dtypes}")

Informações --> Linhas x Colunas: (7499, 18)
---------------------------------------
Informações --> Memória Ocupada: 0.97 MB
---------------------------------------
Informações --> Tipos:
fabrica                      object
linha                         int32
maquina_id                   object
turno                        object
status                       object
data_registro        datetime64[ns]
hora_registro                object
motivo                       object
equipamento                  object
problema                     object
causa                        object
os_numero                    object
operador_id                  object
data_registro_ihm    datetime64[ns]
hora_registro_ihm            object
data_hora            datetime64[ns]
data_hora_final      datetime64[ns]
tempo                         int32
dtype: object


In [89]:
print(f"Produção --> Linhas x Colunas: {df_info_production_cleaned.shape}")
print("---------------------------------------")
print(
    f"Produção --> Memória Ocupada: {df_info_production_cleaned.memory_usage().sum() / 1024 ** 2:.2f} MB"
)
print("---------------------------------------")
print(f"Produção --> Tipos:\n{df_info_production_cleaned.dtypes}")

Produção --> Linhas x Colunas: (888, 8)
---------------------------------------
Produção --> Memória Ocupada: 0.05 MB
---------------------------------------
Produção --> Tipos:
fabrica                    object
linha                       int32
maquina_id                 object
turno                      object
total_ciclos              float64
total_produzido           float64
data_registro      datetime64[ns]
hora_registro              object
dtype: object


# Análise de Dados


## Eficiência, Performance e Reparos de Máquina/Linha


In [205]:
# cspell: words producao


class DataAnalysis:
    def __init__(self, df_stops: pd.DataFrame, df_prod: pd.DataFrame):
        self.df_stops = df_stops
        self.df_prod = df_prod

        # Dicionário com descontos de Eficiência
        self.desc_eff = {
            "Troca de Sabor": 15,
            "Troca de Produto": 35,
            "Refeição": 60,
            "Café e Ginástica Laboral": 10,
            "Treinamento": 60,
        }

        # Dicionário com descontos de Performance
        self.desc_perf = {
            "Troca de Sabor": 15,
            "Refeição": 60,
            "Café e Ginástica Laboral": 10,
            "Treinamento": 60,
        }

        # Dicionário com descontos de Reparo
        self.desc_rep = {"Troca de Produto": 35}

        # Dicionário com o que não afeta a eficiência
        self.not_eff = ["Sem Produção", "Backup"]

        # Dicionário com o que não afeta a performance
        self.not_perf = [
            "Sem Produção",
            "Backup",
            "Limpeza para parada de Fábrica",
            "Risco de Contaminação",
            "Parâmetros de Qualidade",
            "Manutenção",
        ]

        # Dicionário com o que afeta o reparo
        self.afeta_rep = ["Manutenção", "Troca de Produtos"]

    def get_discount(
        self,
        df: pd.DataFrame,
        desc_dict: dict[str, int],
        not_dict: list[str],
        indicator: IndicatorType,
    ) -> pd.DataFrame:
        """
        Calcula o desconto de eficiência, performance ou reparo.
        """

        # Cria uma coluna com o desconto padrão
        df["desconto"] = 0

        # Caso o motivo, problema ou causa não afete o indicador, o desconto é igual a tempo
        mask = (
            df[["motivo", "problema", "causa"]]
            .apply(lambda x: x.str.contains("|".join(not_dict), case=False, na=False))
            .any(axis=1)
        )
        df.loc[mask, "desconto"] = 0 if indicator == IndicatorType.REPAIR else df["tempo"]

        # Cria um dict para indicadores
        indicator_dict = {
            IndicatorType.EFFICIENCY: df,
            IndicatorType.PERFORMANCE: df[~mask],
            IndicatorType.REPAIR: df[mask],
        }

        df = indicator_dict[indicator].reset_index(drop=True)

        # Aplica o desconto de acordo com as colunas "motivo" ou "problema" ou "causa"
        for key, value in desc_dict.items():
            mask = (
                df[["motivo", "problema", "causa"]]
                .apply(lambda x: x.str.contains(key, case=False, na=False))
                .any(axis=1)
            )
            df.loc[mask, "desconto"] = value

        # Caso o desconto seja maior que o tempo, o desconto deve ser igual ao tempo
        df.loc[:, "desconto"] = df[["desconto", "tempo"]].min(axis=1)

        # Calcula o excedente
        df.loc[:, "excedente"] = (df["tempo"] - df["desconto"]).clip(lower=0)

        return df

    def __get_elapsed_time(self, turno: str) -> int:
        """
        Calcula o tempo decorrido do turno.
        """

        # Agora
        now = datetime.now()

        if turno == "MAT" and 8 <= now.hour < 16:
            elapsed_time = now - datetime(now.year, now.month, now.day, 8, 0, 0)
        elif turno == "VES" and 16 <= now.hour < 24:
            elapsed_time = now - datetime(now.year, now.month, now.day, 16, 0, 0)
        elif turno == "NOT" and 0 <= now.hour < 8:
            elapsed_time = now - datetime(now.year, now.month, now.day, 0, 0, 0)
        else:
            return 480

        return elapsed_time.total_seconds() / 60

    def __get_expected_production_time(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Calcula o tempo esperado de produção.
        """

        df["tempo_esperado"] = df.apply(
            lambda row: (
                np.floor(self.__get_elapsed_time(row["turno"]) - row["desconto"])
                if row["data_registro"].date() == pd.to_datetime("today").date()
                else 480 - row["desconto"]
            ),
            axis=1,
        )

        return df

    def get_eff_data(self) -> pd.DataFrame:
        """
        Calcula o desconto de eficiência.
        """

        df = self.df_stops
        df_prod = self.df_prod

        # Calcula o desconto de eficiência
        df = self.get_discount(df, self.desc_eff, self.not_eff, IndicatorType.EFFICIENCY)
        ciclo_ideal = 10.6

        # Agrupa para ter o valor total de desconto
        df = (
            df.groupby(["maquina_id", "linha", "data_registro", "turno"], observed=False)
            .agg(
                tempo=("tempo", "sum"),
                desconto=("desconto", "sum"),
                excedente=("excedente", "sum"),
            )
            .reset_index()
        )

        # Une os dois dataframes
        df = pd.merge(df_prod, df, on=["maquina_id", "linha", "data_registro", "turno"], how="left")

        # Lida com valores nulos
        df = df.fillna(0)

        # Nova coluna para tempo esperado de produção
        df = self.__get_expected_production_time(df)

        # Nova coluna para produção esperada
        df["producao_esperada"] = (df["tempo_esperado"] * ciclo_ideal) * 2

        # Nova coluna para eficiência
        df["eficiencia"] = (df["total_produzido"] / df["producao_esperada"]).round(3)

        # Corrige valores nulos ou inf de eficiência
        df["eficiencia"] = df["eficiencia"].replace([np.inf, -np.inf], np.nan).fillna(0)

        # Se a produção esperada for 0 e a eficiência for 0, tornar a eficiência np.nan
        mask = (df["producao_esperada"] == 0) & (df["eficiencia"] == 0)
        df.loc[mask, "eficiencia"] = np.nan

        # Ordenar as colunas
        df = df[
            [
                "fabrica",
                "linha",
                "maquina_id",
                "turno",
                "data_registro",
                "hora_registro",
                "tempo",
                "total_produzido",
                "producao_esperada",
                "tempo_esperado",
                "desconto",
                "excedente",
                "eficiencia",
            ]
        ]

        return df

    def get_perf_data(self) -> pd.DataFrame:
        """
        Calcula o desconto de performance.
        """

        df = self.df_stops
        df_prod = self.df_prod

        # Dataframe com datas e turnos onde a causa está em not_eff e o tempo é igual a 480
        mask = (df["causa"].isin(["Sem Produção", "Backup"])) & (df["tempo"] == 480)
        paradas_programadas = df[mask][
            ["data_registro", "turno", "linha"]
        ]  # Para performance ser np.nan

        # Calcula o desconto de performance e filtra para remover linhas que não afetam a performance
        df = self.get_discount(df, self.desc_perf, self.not_perf, IndicatorType.PERFORMANCE)

        # Agrupa para ter o valor total de desconto
        df = (
            df.groupby(["maquina_id", "linha", "data_registro", "turno"], observed=False)
            .agg(
                tempo=("tempo", "sum"),
                desconto=("desconto", "sum"),
                afeta=("excedente", "sum"),
            )
            .reset_index()
        )

        # Une os dois dataframes
        df = pd.merge(df_prod, df, on=["maquina_id", "linha", "data_registro", "turno"], how="left")

        # Lida com valores nulos
        df = df.fillna(0)

        # Coluna com tempo esperado de produção
        df = self.__get_expected_production_time(df)

        # Coluna de Performance
        df["performance"] = (df["afeta"] / df["tempo_esperado"]).round(2)

        # Corrige valores nulos ou inf de performance
        df["performance"] = df["performance"].replace([np.inf, -np.inf], np.nan).fillna(0)

        # Criar uma coluna de controle em paradas_programadas
        paradas_programadas["programada"] = True

        # Merge dos dataframes
        df = pd.merge(df, paradas_programadas, how="left", on=["data_registro", "turno", "linha"])

        # Se a parada for programada, a performance é np.nan
        df.loc[df["programada"] is True, "performance"] = np.nan
        df.loc[df["programada"] is True, "tempo_esperado"] = 0

        # Remove a coluna de controle
        df = df.drop(columns="programada")

        # Ordenar as colunas
        df = df[
            [
                "fabrica",
                "linha",
                "maquina_id",
                "turno",
                "data_registro",
                "hora_registro",
                "tempo",
                "tempo_esperado",
                "desconto",
                "afeta",
                "performance",
            ]
        ]

        return df

    def get_repair_data(self) -> pd.DataFrame:
        """
        Calcula o desconto de reparo.
        """

        df = self.df_stops
        df_prod = self.df_prod

        # Dataframe com datas e turnos onde a causa está em not_eff e o tempo é igual a 480
        mask = (df["causa"].isin(["Sem Produção", "Backup"])) & (df["tempo"] == 480)
        paradas_programadas = df[mask][
            ["data_registro", "turno", "linha"]
        ]  # Para reparo ser np.nan

        # Calcula o desconto de reparo
        df = self.get_discount(df, self.desc_rep, self.afeta_rep, IndicatorType.REPAIR)

        # Agrupa para ter o valor total de desconto
        df = (
            df.groupby(["maquina_id", "linha", "data_registro", "turno"], observed=False)
            .agg(
                tempo=("tempo", "sum"),
                desconto=("desconto", "sum"),
                afeta=("excedente", "sum"),
            )
            .reset_index()
        )

        # Une os dois dataframes
        df = pd.merge(df_prod, df, on=["maquina_id", "linha", "data_registro", "turno"], how="left")

        # Lida com valores nulos
        df = df.fillna(0)

        # Coluna com tempo esperado de produção
        df = self.__get_expected_production_time(df)

        # Coluna de Reparo
        df["reparo"] = (df["afeta"] / df["tempo_esperado"]).round(2)

        # Corrige valores nulos ou inf de reparo
        df["reparo"] = df["reparo"].replace([np.inf, -np.inf], np.nan).fillna(0)

        # Criar uma coluna de controle em paradas_programadas
        paradas_programadas["programada"] = True

        # Merge dos dataframes
        df = pd.merge(df, paradas_programadas, how="left", on=["data_registro", "turno", "linha"])

        # Se a parada for programada, a performance é np.nan
        df.loc[df["programada"] is True, "reparo"] = np.nan
        df.loc[df["programada"] is True, "tempo_esperado"] = 0

        # Remove a coluna de controle
        df = df.drop(columns="programada")

        # Ordenar as colunas
        df = df[
            [
                "fabrica",
                "linha",
                "maquina_id",
                "turno",
                "data_registro",
                "hora_registro",
                "tempo",
                "tempo_esperado",
                "desconto",
                "afeta",
                "reparo",
            ]
        ]

        return df


data_analysis = DataAnalysis(df_maq_stopped.copy(), df_info_production_cleaned.copy())

## Saídas de Eficiência, Performance e Reparos de Máquina/Linha


In [148]:
df_eff = data_analysis.get_eff_data()
df_eff

,fabrica,linha,maquina_id,turno,data_registro,hora_registro,tempo,total_produzido,producao_esperada,tempo_esperado,desconto,excedente,eficiencia
0,1,1,TMF003,NOT,2024-05-02,07:59:12,176.0,6274.0,10176.0,480.0,0.0,176.0,0.617
1,1,1,TMF003,MAT,2024-05-02,15:59:13,304.0,2116.0,10176.0,480.0,0.0,304.0,0.208
2,1,1,TMF003,VES,2024-05-02,23:59:15,176.0,5960.0,10176.0,480.0,0.0,176.0,0.586
3,1,1,TMF003,NOT,2024-05-03,07:59:16,82.0,8712.0,10176.0,480.0,0.0,82.0,0.856
4,1,1,TMF003,MAT,2024-05-03,15:59:17,336.0,4000.0,10176.0,480.0,0.0,336.0,0.393
...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,2,14,TMF009,NOT,2024-05-27,07:59:24,480.0,0.0,10176.0,480.0,0.0,480.0,0.000
884,2,14,TMF009,MAT,2024-05-27,15:59:26,480.0,0.0,10176.0,480.0,0.0,480.0,0.000
885,2,14,TMF009,VES,2024-05-27,23:59:27,480.0,0.0,10176.0,480.0,0.0,480.0,0.000
886,2,14,TMF009,NOT,2024-05-28,07:59:29,0.0,0.0,10176.0,480.0,0.0,0.0,0.000


In [206]:
df_perf = data_analysis.get_perf_data()
df_perf

Index(['fabrica', 'linha', 'maquina_id', 'turno', 'total_ciclos',
       'total_produzido', 'data_registro', 'hora_registro', 'tempo',
       'desconto', 'afeta', 'tempo_esperado', 'performance', 'programada'],
      dtype='object')


,fabrica,linha,maquina_id,turno,data_registro,hora_registro,tempo,tempo_esperado,desconto,afeta,performance
0,1,1,TMF003,NOT,2024-05-02,07:59:12,176.0,480.0,0.0,176.0,0.37
1,1,1,TMF003,MAT,2024-05-02,15:59:13,304.0,480.0,0.0,304.0,0.63
2,1,1,TMF003,VES,2024-05-02,23:59:15,176.0,480.0,0.0,176.0,0.37
3,1,1,TMF003,NOT,2024-05-03,07:59:16,82.0,480.0,0.0,82.0,0.17
4,1,1,TMF003,MAT,2024-05-03,15:59:17,336.0,480.0,0.0,336.0,0.70
...,...,...,...,...,...,...,...,...,...,...,...
883,2,14,TMF009,NOT,2024-05-27,07:59:24,480.0,480.0,0.0,480.0,1.00
884,2,14,TMF009,MAT,2024-05-27,15:59:26,480.0,480.0,0.0,480.0,1.00
885,2,14,TMF009,VES,2024-05-27,23:59:27,480.0,480.0,0.0,480.0,1.00
886,2,14,TMF009,NOT,2024-05-28,07:59:29,0.0,480.0,0.0,0.0,0.00


In [150]:
df_repair = data_analysis.get_repair_data()
df_repair

,fabrica,linha,maquina_id,turno,data_registro,hora_registro,tempo,tempo_esperado,desconto,afeta,reparo
0,1,1,TMF003,NOT,2024-05-02,07:59:12,0.0,480.0,0.0,0.0,0.0
1,1,1,TMF003,MAT,2024-05-02,15:59:13,0.0,480.0,0.0,0.0,0.0
2,1,1,TMF003,VES,2024-05-02,23:59:15,0.0,480.0,0.0,0.0,0.0
3,1,1,TMF003,NOT,2024-05-03,07:59:16,0.0,480.0,0.0,0.0,0.0
4,1,1,TMF003,MAT,2024-05-03,15:59:17,0.0,480.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
883,2,14,TMF009,NOT,2024-05-27,07:59:24,0.0,480.0,0.0,0.0,0.0
884,2,14,TMF009,MAT,2024-05-27,15:59:26,0.0,480.0,0.0,0.0,0.0
885,2,14,TMF009,VES,2024-05-27,23:59:27,0.0,480.0,0.0,0.0,0.0
886,2,14,TMF009,NOT,2024-05-28,07:59:29,0.0,480.0,0.0,0.0,0.0


# Dataframes auxiliares para uso dos gráficos


In [151]:
class DFIndicators:
    def __init__(self, df_info_ihm: pd.DataFrame, df_prod: pd.DataFrame):
        self.df_info_ihm = df_info_ihm
        self.df_prod = df_prod
        self.data_analysis = DataAnalysis(self.df_info_ihm, self.df_prod)
        self.indicator_functions = {
            IndicatorType.EFFICIENCY: self.data_analysis.get_eff_data,
            IndicatorType.PERFORMANCE: self.data_analysis.get_perf_data,
            IndicatorType.REPAIR: self.data_analysis.get_repair_data,
        }
        self.indicator_descontos = {
            IndicatorType.EFFICIENCY: self.data_analysis.desc_eff,
            IndicatorType.PERFORMANCE: self.data_analysis.desc_perf,
            IndicatorType.REPAIR: self.data_analysis.desc_rep,
        }
        self.indicator_not_affect = {
            IndicatorType.EFFICIENCY: self.data_analysis.not_eff,
            IndicatorType.PERFORMANCE: self.data_analysis.not_perf,
            IndicatorType.REPAIR: self.data_analysis.afeta_rep,
        }

    def __adjust_heatmap_data(
        self, indicator: IndicatorType, turn: str = None, main: bool = False
    ) -> pd.DataFrame:
        """
        Adjusts the heatmap data based on the given indicator, turn, and main flag.

        Parameters:
            indicator (IndicatorType): The indicator type to adjust the data for.
            turn (str, optional): The turn to filter the data for. Defaults to None.
            main (bool, optional): Flag indicating whether the main grouping should be used. Defaults to False.

        Returns:
            pd.DataFrame: The adjusted dataframe with the heatmap data.
        """

        # Dataframe com os dados do indicador
        df = pd.DataFrame()

        # Busca o dataframe de acordo com o indicador
        df = self.indicator_functions[indicator]()

        # Se for indicado um turno, filtra o dataframe
        if turn:
            df = df[df["turno"] == turn]

        # Colunas para agrupar o dataframe
        group_cols = ["data_registro", "turno"] if main else ["data_registro", "linha"]

        # Agrupa o dataframe e calcula a média
        df = df.groupby(group_cols, observed=False)[indicator.value].mean().reset_index()

        # ====================== Garantir Que Todas Datas Estejam Presentes ====================== #

        # Obter a data do inicio e fim do mês atual
        start_date = pd.to_datetime("today").replace(day=1).date()
        end_date = (pd.to_datetime("today") + pd.offsets.MonthEnd(0)).date()

        # Lista com todas as datas do mês atual
        date_range = pd.date_range(start_date, end_date, freq="D").date

        # Encontra os valores únicos de turno ou linha e cria um novo index com todas as datas e turnos ou linhas
        if main:
            unique_turnos = df["turno"].unique()
            df.set_index(["data_registro", "turno"], inplace=True)
            new_index = pd.MultiIndex.from_product(
                [date_range, unique_turnos], names=["data_registro", "turno"]
            )
        else:
            unique_lines = df["linha"].unique()
            df.set_index(["data_registro", "linha"], inplace=True)
            new_index = pd.MultiIndex.from_product(
                [date_range, unique_lines], names=["data_registro", "linha"]
            )

        # Reindexa o dataframe
        df = df.reindex(new_index).reset_index()

        # ================================== Pivotar O Dataframe ================================= #

        # Pivotar o dataframe
        if main:
            df = df.pivot(index="turno", columns="data_registro", values=indicator.value)
            df = df.reindex(["NOT", "MAT", "VES"])
        else:
            df = df.pivot(index="linha", columns="data_registro", values=indicator.value)

        return df

    def get_heatmap_data(self, indicator: IndicatorType) -> tuple:
        """
        Retrieves heatmap data for the given indicator.

        Args:
            indicator (IndicatorType): The indicator type to retrieve data for.

        Returns:
            tuple: A tuple containing the heatmap data for different shifts:
                - noturno: Heatmap data for the night shift.
                - matutino: Heatmap data for the morning shift.
                - vespertino: Heatmap data for the afternoon shift.
                - main: Heatmap data for the main shift.
        """

        noturno = self.__adjust_heatmap_data(indicator, "NOT")
        matutino = self.__adjust_heatmap_data(indicator, "MAT")
        vespertino = self.__adjust_heatmap_data(indicator, "VES")
        main = self.__adjust_heatmap_data(indicator, main=True)

        return noturno, matutino, vespertino, main

    @staticmethod
    def __annotations_list(df: pd.DataFrame) -> list:
        """
        Cria uma lista de anotações para o heatmap.

        Args:
            df (pd.DataFrame): O dataframe com os dados do heatmap.

        Returns:
            list: Uma lista de anotações para o heatmap.
        """

        # Inicializa a lista de anotações
        annotations = []

        # Define a coluna para dia apenas
        df.columns = pd.to_datetime(df.columns).day

        # Loop sobre as linhas do dataframe
        for (i, j), value in np.ndenumerate(df.values):
            # Se o valor for diferente de NaN
            if not np.isnan(value):
                # Adiciona a anotação
                annotations.append(
                    {
                        "x": df.columns[j],
                        "y": df.index[i],
                        "text": f"{value:.1%}",
                        "xref": "x",
                        "yref": "y",
                        "showarrow": False,
                        "font": {"size": 10, "color": "white"},
                    }
                )

        return annotations

    def get_annotations(self, indicator: IndicatorType) -> tuple:
        """
        Retrieves annotations for the heatmap data.

        Args:
            indicator (IndicatorType): The indicator type to retrieve annotations for.

        Returns:
            tuple: A tuple containing the annotations for different shifts:
                - noturno: Annotations for the night shift.
                - matutino: Annotations for the morning shift.
                - vespertino: Annotations for the afternoon shift.
                - main: Annotations for the main shift.
        """

        noturno, matutino, vespertino, main = self.get_heatmap_data(indicator)

        noturno_annotations = self.__annotations_list(noturno)
        matutino_annotations = self.__annotations_list(matutino)
        vespertino_annotations = self.__annotations_list(vespertino)
        main_annotations = self.__annotations_list(main)

        return noturno_annotations, matutino_annotations, vespertino_annotations, main_annotations

    def adjust_df_for_bar_lost(
        self,
        df: pd.DataFrame,
        indicator: IndicatorType,
        turn: str = "TOT",
        working_minutes: pd.DataFrame = None,
    ) -> pd.DataFrame:

        # Cria dataframe com os tempos
        df = self.data_analysis.get_discount(
            df, self.indicator_descontos[indicator], self.indicator_not_affect[indicator], indicator
        )

        # Une com working_minutes
        if working_minutes is not None:
            df = pd.concat([df, working_minutes], ignore_index=True, sort=False)

        # Filtra por turno
        df = df[df["turno"] == turn] if turn != "TOT" else df

        # Lidando com paradas de  5 minutos ou menos
        mask = (df["motivo"].isnull()) & (df["tempo"] <= 5)
        columns_to_fill = ["motivo", "problema", "causa"]
        fill_value = "Parada de 5 minutos ou menos"

        for column in columns_to_fill:
            df.loc[mask, column] = fill_value

        # Lidando com motivo nulo
        mask = mask = df["motivo"].isnull()
        columns_to_fill = ["motivo", "problema", "causa"]
        fill_value = "Não apontado"

        for column in columns_to_fill:
            df.loc[mask, column] = fill_value

        return df


indicators_class = DFIndicators(df_maq_stopped.copy(), df_info_production_cleaned.copy())

In [152]:
heat_noturno, heat_matutino, heat_vespertino, heat_main = indicators_class.get_heatmap_data(
    IndicatorType.PERFORMANCE
)
heat_noturno
heat_matutino
heat_vespertino

     data_registro turno  linha
671     2024-05-17   NOT      3
686     2024-05-21   NOT      3
705     2024-05-23   NOT      3
777     2024-05-26   NOT      3
1460    2024-05-26   NOT      5
1469    2024-05-27   NOT      5
1672    2024-05-15   MAT      6
1782    2024-05-25   VES      6
2519    2024-05-24   MAT      8
2522    2024-05-25   MAT      8
2524    2024-05-26   NOT      8
2844    2024-05-21   NOT      9
2847    2024-05-22   NOT      9
     data_registro turno  linha
671     2024-05-17   NOT      3
686     2024-05-21   NOT      3
705     2024-05-23   NOT      3
777     2024-05-26   NOT      3
1460    2024-05-26   NOT      5
1469    2024-05-27   NOT      5
1672    2024-05-15   MAT      6
1782    2024-05-25   VES      6
2519    2024-05-24   MAT      8
2522    2024-05-25   MAT      8
2524    2024-05-26   NOT      8
2844    2024-05-21   NOT      9
2847    2024-05-22   NOT      9
     data_registro turno  linha
671     2024-05-17   NOT      3
686     2024-05-21   NOT      3
705     

data_registro,2024-05-01,2024-05-02,2024-05-03,2024-05-04,2024-05-05,2024-05-06,2024-05-07,2024-05-08,2024-05-09,2024-05-10,...,2024-05-22,2024-05-23,2024-05-24,2024-05-25,2024-05-26,2024-05-27,2024-05-28,2024-05-29,2024-05-30,2024-05-31
linha,,,,,,,,,,,,,,,,,,,,,
1,NaN,0.37,0.34,1.02,NaN,0.49,0.30,0.30,0.39,0.23,...,0.20,0.12,0.15,0.08,0.07,0.04,NaN,NaN,NaN,NaN
2,NaN,0.00,0.00,1.62,NaN,0.02,0.02,0.01,0.02,0.00,...,0.02,0.39,0.07,0.04,0.08,0.01,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,1.00,1.00,1.00,1.00,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0.37,0.36,1.05,NaN,0.67,0.34,0.35,0.41,0.29,...,0.26,0.17,0.29,0.81,0.56,0.08,NaN,NaN,NaN,NaN
5,NaN,0.28,0.28,1.01,NaN,0.06,0.30,0.21,0.08,0.32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,0.33,0.18,1.00,NaN,0.05,0.16,0.38,0.25,0.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,0.05,0.11,1.16,NaN,1.00,1.00,0.32,0.34,0.07,...,0.32,0.34,0.32,0.07,0.06,0.19,NaN,NaN,NaN,NaN
8,NaN,0.96,1.00,1.00,NaN,0.72,0.25,0.20,0.23,0.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,0.61,0.40,1.14,NaN,0.41,0.35,0.96,1.00,0.94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
heat_main

data_registro,2024-05-01,2024-05-02,2024-05-03,2024-05-04,2024-05-05,2024-05-06,2024-05-07,2024-05-08,2024-05-09,2024-05-10,...,2024-05-22,2024-05-23,2024-05-24,2024-05-25,2024-05-26,2024-05-27,2024-05-28,2024-05-29,2024-05-30,2024-05-31
turno,,,,,,,,,,,,,,,,,,,,,
NOT,NaN,0.583846,0.503846,0.529231,NaN,0.545000,0.555385,0.505714,0.556429,0.525000,...,0.428889,0.432222,0.402222,0.418889,0.380000,0.383333,0.134286,NaN,NaN,NaN
MAT,NaN,0.440769,0.541538,0.446923,NaN,0.523846,0.487143,0.427143,0.432857,0.457143,...,0.401111,0.397778,0.421111,0.315556,0.464444,0.313333,0.082857,NaN,NaN,NaN
VES,NaN,0.536923,0.464615,1.076923,NaN,0.592308,0.500000,0.515000,0.527143,0.515714,...,0.506667,0.548889,0.520000,0.444444,0.466667,0.371111,NaN,NaN,NaN,NaN


In [154]:
annotations_noturno, _, _, _ = indicators_class.get_annotations(IndicatorType.EFFICIENCY)
annotations_noturno

[{'x': 2,
  'y': 1,
  'text': '61.7%',
  'xref': 'x',
  'yref': 'y',
  'showarrow': False,
  'font': {'size': 10, 'color': 'white'}},
 {'x': 3,
  'y': 1,
  'text': '85.6%',
  'xref': 'x',
  'yref': 'y',
  'showarrow': False,
  'font': {'size': 10, 'color': 'white'}},
 {'x': 4,
  'y': 1,
  'text': '76.5%',
  'xref': 'x',
  'yref': 'y',
  'showarrow': False,
  'font': {'size': 10, 'color': 'white'}},
 {'x': 6,
  'y': 1,
  'text': '79.8%',
  'xref': 'x',
  'yref': 'y',
  'showarrow': False,
  'font': {'size': 10, 'color': 'white'}},
 {'x': 7,
  'y': 1,
  'text': '88.0%',
  'xref': 'x',
  'yref': 'y',
  'showarrow': False,
  'font': {'size': 10, 'color': 'white'}},
 {'x': 8,
  'y': 1,
  'text': '86.6%',
  'xref': 'x',
  'yref': 'y',
  'showarrow': False,
  'font': {'size': 10, 'color': 'white'}},
 {'x': 9,
  'y': 1,
  'text': '79.6%',
  'xref': 'x',
  'yref': 'y',
  'showarrow': False,
  'font': {'size': 10, 'color': 'white'}},
 {'x': 10,
  'y': 1,
  'text': '85.6%',
  'xref': 'x',
  'yref

# Gráficos


## Gauge


In [155]:
class Gauge:

    def __init__(self):
        self.danger = BSColorsEnum.DANGER_COLOR.value
        self.success = BSColorsEnum.SUCCESS_COLOR.value

    def create_gauge(
        self,
        df: pd.DataFrame,
        indicator: IndicatorType,
        meta: int,
        template: str = None,
        this_month: bool = True,
    ):

        month = "Mês Atual" if this_month else "Mês Anterior"

        # Calcula a porcentagem de acordo com o indicador
        percentage = df[indicator.value].mean() if this_month else df[indicator.value].iloc[-1]
        percentage = percentage if percentage is not None else 0

        # Arredonda o valor da porcentagem
        percentage = round(percentage, 2)

        # Define a cor de acordo com a porcentagem
        color = self.danger if percentage > (meta / 100) else self.success

        if indicator == IndicatorType.EFFICIENCY:
            color = self.success if percentage >= (meta / 100) else self.danger

        # Definis a escala
        axis_range = [0, 100] if indicator == IndicatorType.EFFICIENCY else [40, 0]

        # Criar o gráfico
        figure = go.Figure(
            go.Indicator(
                mode="gauge+number",
                value=percentage * 100,
                number={"suffix": "%"},
                domain={"x": [0, 1], "y": [0, 1]},
                title={"text": f"{month}", "font": {"size": 14}},
                gauge={
                    "axis": {
                        "range": axis_range,
                        "tickfont": {"size": 8},
                    },
                    "bar": {"color": color},
                    "steps": [
                        {"range": [0, 100], "color": "lightgray"},
                    ],
                    "threshold": {
                        "line": {"color": "black", "width": 2},
                        "thickness": 0.75,
                        "value": meta,
                    },
                },
            )
        )

        figure.update_layout(
            autosize=True,
            margin=dict(t=30, b=30, l=30, r=30),
            plot_bgcolor="white",
            height=250,
            font=dict({"family": "Inter"}),
        )

        return figure


gauge = Gauge()

In [156]:
gauge_fig = gauge.create_gauge(df_eff, IndicatorType.EFFICIENCY, 85)
gauge_fig

## Heatmap


In [157]:
class Heatmap:
    """
    A class representing a heatmap graph.

    Attributes:
        danger (str): The color value for the danger indicator.
        success (str): The color value for the success indicator.

    Methods:
        create_heatmap: Create a heatmap graph based on the provided data.
    """

    def __init__(self):
        self.danger = BSColorsEnum.DANGER_COLOR.value
        self.success = BSColorsEnum.SUCCESS_COLOR.value

    def create_heatmap(
        self,
        dataframe: pd.DataFrame,
        annotations: list,
        indicator: IndicatorType,
        meta: int,
        template: str = None,
        turn: str = False,
    ):
        """
        Create a heatmap graph based on the provided data.

        Args:
            dataframe (pd.DataFrame): The data to be visualized in the heatmap.
            annotations (list): List of annotations to be displayed on the heatmap.
            indicator (IndicatorType): The type of indicator to be visualized.
            meta (int): The meta value for the indicator.
            template (str, optional): The template to be used for the graph. Defaults to None.
            turn (str, optional): Whether to display the heatmap by turn or not. Defaults to False.

        Returns:
            dcc.Graph: The heatmap graph.
        """

        # Meta p/ uso em cores
        color_meta = meta / 100

        # Cria escala de cores
        color_scale = {
            IndicatorType.EFFICIENCY: [
                [0, self.danger],
                [color_meta, self.danger],
                [color_meta, self.success],
                [1, self.success],
            ],
            IndicatorType.PERFORMANCE: [
                [0, self.success],
                [color_meta, self.success],
                [color_meta, self.danger],
                [1, self.danger],
            ],
            IndicatorType.REPAIR: [
                [0, self.success],
                [color_meta, self.success],
                [color_meta, self.danger],
                [1, self.danger],
            ],
        }

        # Extrai apenas o dia da data
        dataframe.columns = pd.to_datetime(dataframe.columns).day

        # Cria o hover data
        hover_data = (
            f"Turno: %{{y}}<br>Dia: %{{x}}<br>{indicator.value.capitalize()}: %{{z:.1%}}"
            if not turn
            else f"Linha: %{{y}}<br>Dia: %{{x}}<br>{indicator.value.capitalize()}: %{{z:.1%}}"
        )

        tick_color = "gray"

        # Cria o heatmap
        figure = go.Figure(
            data=go.Heatmap(
                z=dataframe.values,
                x=dataframe.columns,
                y=dataframe.index,
                colorscale=color_scale[indicator],
                name=indicator.value.capitalize(),
                zmin=0,
                zmax=1,
                hoverongaps=False,
                hovertemplate=hover_data,
                showscale=False,
                xgap=1,
                ygap=1,
            ),
            layout=go.Layout(
                title=f"{indicator.value.capitalize()} - Meta: {meta}%",
                title_x=0.5,
                xaxis=dict(
                    title="Dia do Mês",
                    tickfont=dict(color=tick_color),
                    tickmode="linear",
                    nticks=31,
                    tickvals=list(range(1, 32)),
                    ticktext=list(range(1, 32)),
                    tickangle=0,
                ),
                yaxis=dict(title="Turno", tickfont=dict(color=tick_color), ticksuffix=" "),
                font=dict(family="Inter"),
                margin=dict(t=40, b=40, l=40, r=40),
                annotations=annotations,
                plot_bgcolor="RGBA(0,0,0,0.01)",
            ),
        )

        if turn:
            figure.update_layout(
                yaxis=dict(
                    title="Linha",
                    autorange="reversed",
                    tickvals=list(range(1, dataframe.index.nunique() + 1)),
                )
            )

        return figure


heatmap = Heatmap()

In [158]:
heatmap_fig = heatmap.create_heatmap(
    heat_noturno, annotations_noturno, IndicatorType.PERFORMANCE, 4, turn=True
)
heatmap_fig

## Line


In [159]:
class LineGraph:
    """
    A class representing a line graph.

    Attributes:
        secondary (str): The secondary color for the graph.
        primary (str): The primary color for the graph.
        danger (str): The danger color for the graph.

    Methods:
        create_line_graph(dataframe, indicator, meta, template, turn=None):
            Create a line graph based on the provided data.

    """

    def __init__(self):
        self.secondary = BSColorsEnum.SECONDARY_COLOR.value
        self.primary = BSColorsEnum.PRIMARY_COLOR.value
        self.danger = BSColorsEnum.DANGER_COLOR.value

    def create_line_graph(
        self,
        dataframe: pd.DataFrame,
        indicator: IndicatorType,
        meta: int,
        turn: str = None,
    ):
        """
        Create a line graph based on the provided data.

        Args:
            dataframe (pd.DataFrame): The input data containing the indicator values.
            indicator (IndicatorType): The type of indicator to be plotted.
            meta (int): The target value for the indicator.
            template (TemplateType): The template type for the graph.
            turn (str, optional): The specific turn to filter the data. Defaults to None.

        Returns:
            dcc.Graph: The line graph visualizing the indicator values.

        """

        # Pegar o valor do indicador
        indicator = indicator.value

        # Converter 'data_registro' para datetime e criar uma nova coluna 'data_turno'
        dataframe["data_registro"] = pd.to_datetime(dataframe["data_registro"])
        dataframe["data_turno"] = dataframe["data_registro"].dt.strftime("%Y-%m-%d")

        # Filtrar por turno
        dataframe = dataframe[dataframe["turno"] == turn] if turn and turn != "TOT" else dataframe

        # Agrupar por 'data_turno' e 'turno' e calcular a média do indicador
        df_grouped = dataframe.groupby(["data_turno"])[indicator].mean().reset_index()

        # Multiplicar por 100 para converter para porcentagem
        df_grouped[indicator] = df_grouped[indicator] * 100

        # Criação de um DataFrame com todas as datas do mês atual
        start_date = dataframe["data_registro"].min()
        end_date = start_date + pd.tseries.offsets.MonthEnd(1)
        all_dates = pd.date_range(start_date, end_date).strftime("%Y-%m-%d")
        dataframe_all_dates = pd.DataFrame(all_dates, columns=["data_turno"])

        # Mesclar o DataFrame original com o DataFrame de todas as datas
        df_grouped = pd.merge(dataframe_all_dates, df_grouped, on="data_turno", how="left")

        # Substituir valores nulos por 0
        df_grouped[indicator] = df_grouped[indicator].fillna(0)

        # Criação do gráfico
        fig = go.Figure(
            data=[
                go.Scatter(
                    x=df_grouped["data_turno"],
                    y=df_grouped[indicator],
                    mode="lines+markers",
                    marker=dict(color=self.primary),
                    line=dict(color=self.primary),
                    hovertemplate=f"<i>Dia</i>: %{{x}}"
                    f"<br><b>{indicator.capitalize()}</b>: %{{y:.1f}}<br>",
                    hoverinfo="skip",
                )
            ],
            layout=go.Layout(
                showlegend=False,
                xaxis=dict(showticklabels=False),
                yaxis=dict(showticklabels=False, autorange=True),
                margin=dict(l=0, r=0, t=0, b=0),
                height=None,
                autosize=True,
                font=dict(family="Inter"),
                plot_bgcolor="RGBA(0,0,0,0.001)",
            ),
        )

        # Linha de meta
        fig.add_trace(
            go.Scatter(
                x=df_grouped["data_turno"],
                y=[meta] * len(df_grouped),
                mode="lines",
                name="Meta",
                line=dict(color=self.danger, dash="dash"),
                hovertemplate="<b>Meta</b>: %{y:.0f}%<extra></extra>",
            )
        )

        return fig


line_graph = LineGraph()

In [160]:
line_fig = line_graph.create_line_graph(df_eff, IndicatorType.EFFICIENCY, 85, "NOT")
line_fig

## Bar Chart


In [161]:
class BarChartDetails:
    """
    Represents a class for creating bar chart details based on provided data.
    """

    def __init__(self, df_maq_stopped: pd.DataFrame, df_production: pd.DataFrame):
        self.df_indicator = DFIndicators(df_maq_stopped, df_production)

    def create_bar_chart_details(
        self,
        dataframe: pd.DataFrame,
        indicator: IndicatorType,
        turn: str = None,
        selected_data: str = None,
        working: pd.DataFrame = None,
        choice: str = "motivo",
    ):
        """
        Creates a bar chart with details based on the provided data.

        Args:
            dataframe (pd.DataFrame): The input dataframe containing the data.
            indicator (IndicatorType): The type of indicator.
            template (TemplateType): The type of template.
            turn (str): The turn value.
            selected_data: The selected data.
            working (pd.DataFrame, optional): The working dataframe. Defaults to None.

        Returns:
            dcc.Graph: The bar chart as a Dash component.
        """
        df = self.df_indicator.adjust_df_for_bar_lost(dataframe, indicator, working_minutes=working)

        # Garante que data_registro tenha apenas o dia
        df["data_registro"] = pd.to_datetime(df["data_registro"]).dt.date

        # Filtro pelo dia, caso tenha sido selecionado
        df = (
            df[df["data_registro"] == pd.to_datetime(selected_data).date()]
            if selected_data is not None
            else df
        )

        motivos = df[choice].unique()
        # Definir cores
        palette = sns.color_palette("tab20", len(motivos)).as_hex()

        color_dict = dict(zip(motivos, palette))

        if choice == "motivo":
            # Criar coluna sort para ordenar os dados
            df["sort"] = (
                df["motivo"].map({"Rodando": 0, "Parada Programada": 2, "Limpeza": 1}).fillna(9)
            )

            # Ordenar por sort e tempo_registro_min
            df = df.sort_values(
                by=["linha", "sort", "data_registro", "tempo"],
                ascending=[True, True, True, False],
            )

            # Remover a coluna sort
            df = df.drop(columns=["sort"])

            # Mapear cores
            color_dict = {
                "Parada de 5 minutos ou menos": BSColorsEnum.BLUE_DELFT_COLOR.value,
                "Não apontado": BSColorsEnum.GREY_600_COLOR.value,
                "Ajustes": BSColorsEnum.TEAL_COLOR.value,
                "Manutenção": BSColorsEnum.SPACE_CADET_COLOR.value,
                "Qualidade": BSColorsEnum.WARNING_COLOR.value,
                "Fluxo": BSColorsEnum.PINK_COLOR.value,
                "Parada Programada": BSColorsEnum.DANGER_COLOR.value,
                "Setup": BSColorsEnum.SECONDARY_COLOR.value,
                "Limpeza": BSColorsEnum.PRIMARY_COLOR.value,
                "Rodando": BSColorsEnum.SUCCESS_COLOR.value,
            }

        # Criação do gráfico
        fig = px.bar(
            df,
            x="linha",
            y="tempo",
            color=choice,
            barmode="stack",
            color_discrete_map=color_dict,
            title="Detalhes de Tempo",
            labels={
                "tempo": "Tempo (min)",
                "linha": "Linha",
                choice: choice.capitalize(),
            },
        )

        tick_color = "gray"

        # Atualizar layout
        fig.update_layout(
            xaxis=dict(
                categoryorder="category ascending",
                tickvals=df["linha"].unique(),
                tickfont=dict(color=tick_color),
            ),
            yaxis=dict(tickfont=dict(color=tick_color)),
            title_x=0.5,
            xaxis_title="Linha",
            yaxis_title="Tempo (min)",
            legend_title=choice.capitalize(),
            plot_bgcolor="RGBA(0,0,0,0.01)",
            margin=dict(t=40, b=40, l=40, r=40),
        )

        return fig


bar_chart_details = BarChartDetails(df_maq_stopped.copy(), df_info_production_cleaned.copy())

In [162]:
chart_details = bar_chart_details.create_bar_chart_details(
    df_maq_stopped.copy(), IndicatorType.EFFICIENCY, working=df_time_working.copy()
)
chart_details

In [163]:
chart_details_2 = bar_chart_details.create_bar_chart_details(
    df_maq_stopped.copy(),
    IndicatorType.EFFICIENCY,
    working=df_time_working.copy(),
    choice="problema",
)
chart_details_2

In [164]:
chart_details_3 = bar_chart_details.create_bar_chart_details(
    df_maq_stopped.copy(),
    IndicatorType.EFFICIENCY,
    working=df_time_working.copy(),
    choice="causa",
)
chart_details_3

In [165]:
chart_details_4 = bar_chart_details.create_bar_chart_details(
    df_maq_stopped.copy(),
    IndicatorType.EFFICIENCY,
    working=df_time_working.copy(),
    choice="equipamento",
)
chart_details_4

In [166]:
class SunburstTest:
    """
    Represents a class for creating bar chart details based on provided data.
    """

    def __init__(self, df_maq_stopped: pd.DataFrame, df_production: pd.DataFrame):
        self.df_indicator = DFIndicators(df_maq_stopped, df_production)

    def create_icicle_details(
        self,
        dataframe: pd.DataFrame,
        indicator: IndicatorType,
        turn: str = None,
        selected_data: str = None,
        working: pd.DataFrame = None,
    ):

        df = self.df_indicator.adjust_df_for_bar_lost(dataframe, indicator)

        # Garante que data_registro tenha apenas o dia
        df["data_registro"] = pd.to_datetime(df["data_registro"]).dt.date

        # Filtro pelo dia, caso tenha sido selecionado
        df = (
            df[df["data_registro"] == pd.to_datetime(selected_data).date()] if selected_data else df
        )

        # Remove onde não há motivo informado
        df = df[df["motivo"] != "Não apontado"]

        # Preenche equipamento nulo
        df["equipamento"] = df["equipamento"].fillna("Linha")

        # Se motivo for limpeza, causa deve ser igual ao problema
        df.loc[df["motivo"] == "Limpeza", "causa"] = df["problema"]

        # Mapear cores
        color_dict = {
            "Parada de 5 minutos ou menos": BSColorsEnum.GREY_400_COLOR.value,
            "Sem Motivo Informado": BSColorsEnum.GREY_600_COLOR.value,
            "Ajustes": BSColorsEnum.TEAL_COLOR.value,
            "Manutenção": BSColorsEnum.SPACE_CADET_COLOR.value,
            "Qualidade": BSColorsEnum.WARNING_COLOR.value,
            "Fluxo": BSColorsEnum.PINK_COLOR.value,
            "Parada Programada": BSColorsEnum.DANGER_COLOR.value,
            "Setup": BSColorsEnum.SECONDARY_COLOR.value,
            "Limpeza": BSColorsEnum.PRIMARY_COLOR.value,
            "Rodando": BSColorsEnum.SUCCESS_COLOR.value,
        }

        # Criação do gráfico
        fig = px.icicle(
            df,
            path=[px.Constant("Maquina ID"), "turno", "motivo", "equipamento", "problema", "causa"],
            values="tempo",
            color="motivo",
            title="Detalhes de Tempo",
            height=800,
            color_discrete_map=color_dict,
        )

        tick_color = "gray"

        # Atualizar layout
        fig.update_layout(
            plot_bgcolor="RGBA(0,0,0,0.01)",
        )

        # Adicionar a porcentagem
        fig.update_traces(textinfo="label+percent parent")

        return fig


sunburst_test = SunburstTest(df_maq_stopped.copy(), df_info_production_cleaned.copy())

In [167]:
icicle_test_fig = sunburst_test.create_icicle_details(
    df_maq_stopped.copy(), IndicatorType.EFFICIENCY
)
icicle_test_fig